# Frequency-Accuracy Analysis

This notebook analyze the relationship between the frequency of the signal and the predicted accuracy of the hypnosis depth. This analysis is done response to the following reviewer's comment:

"The fact that higher frequency components contribute more to the prediction of the hypnosis depth suggest a more general relationship between hypnosis and the EEG spectrum. It would have been informative to plot a frequency – accuracy plot where frequency is divided in narrow frequency intervals with a logarithmic granularity from 1 Hz up to 60 Hz on the y-axis and prediction accuracy represented on the x-axis. The deviation of that curve would be interesting to analyze relative to the linear function. Would we see steps with the conventional delta, theta, alpha, beta and gamma bands?"

In [1]:
# imports
import numpy as np
import pandas as pd
import pickle as pkl
import mne

# constants
epochs = mne.read_epochs('data/clean_data/sub-01_ses-01_task-baseline1_proc-clean_epo.fif')
ch_names = epochs.ch_names.copy()  # make sure to copy the list because it is mutable in place
[ch_names.remove(i) for i in ['M1', 'M2', 'EOG1', 'EOG2', 'ECG']]
all_channels = epochs.ch_names
del epochs

# name of electrode groups
ba_patches = {'LF': ['Fp1', 'F3', 'F7', 'AF3', 'F1', 'F5'],
 'LC': ['C3', 'T7', 'FC1', 'FC3', 'FC5', 'C1', 'C5', 'FT7'],
 'LP': ['P3', 'P7', 'CP1', 'CP3', 'CP5', 'TP7', 'P1', 'P5'],
 'LO': ['O1', 'PO3'],
 'RF': ['Fp2', 'F4', 'F8', 'AF4', 'F2', 'F6',],
 'RC': ['C4', 'T8', 'FC2', 'FC4', 'FC6', 'C2', 'C6', 'FT8'],
 'RP': ['P4', 'P8', 'CP2', 'CP4', 'CP6', 'TP8', 'P2', 'P6'],
 'RO': ['O2', 'PO4'],
 'FZ': ['Fpz', 'Fz'],
 'CZ': ['Cz', 'FCz'],
 'PZ': ['Pz', 'CPz'],
 'OZ': ['POz', 'Oz', 'Iz'],
#  'all': ch_names
}

# index of electrode groups
rois = {}
for k,v in ba_patches.items():
    temp = [all_channels.index(i) for i in v]
    rois[k] = temp

Reading /Users/yeganeh/Codes/SugNet/data/clean_data/sub-01_ses-01_task-baseline1_proc-clean_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     999.00 ms
        0 CTF compensation matrices available
Not setting metadata
291 matching events found
No baseline correction applied
0 projection items activated


## Feature extraction

### Power Sensor

In [2]:
# open power sensor
with open(f'docs/psds_3th_higher_frequency_resolution.pkl', 'rb') as f:
    psds = pkl.load(f)


def constant_q_bins(fmin=1.0, fmax=42.0, bins_per_oct=6):
    centers = []
    f = fmin
    while f <= fmax:
        f *= 2**(1/bins_per_oct)
        centers.append(f)
    centers = np.array(centers)
    width = 2**(1/(2*bins_per_oct))
    edges = np.column_stack([centers/width, centers*width])
    edges[:, 0] = np.maximum(edges[:, 0], fmin)
    edges[:, 1] = np.minimum(edges[:, 1], fmax)
    return centers, edges  # (n_bins,), (n_bins,2)


freqs = psds.pop('freqs')

In [3]:
roi_features_all = {}
for key in psds.keys():

    # Interpolate PSD onto dense frequency grid
    psd = psds[key]  # (n_channels, n_freqs)
    freq_dense = np.arange(1, 42, 0.02)
    psd_dense = np.array([np.interp(freq_dense, freqs, psd[i]) for i in range(psd.shape[0])])

    centers, edges = constant_q_bins(1.0, 42.0, bins_per_oct=6)

    # 1) Bin power per channel (physical approach)
    bin_vals = []
    for f1, f2 in edges:
        m = (freq_dense >= f1) & (freq_dense <= f2)
        # integrate linear PSD then divide by bandwidth
        bw = (f2 - f1)
        # trapezoidal integral per channel
        p_lin = np.trapezoid(psd_dense[:, m], freq_dense[m], axis=1) / bw
        p_db  = 10*np.log10(p_lin + 1e-20)  # (n_channels,)
        bin_vals.append(p_lin)
    bin_vals = np.stack(bin_vals, axis=1)  # (n_channels, n_bins)

    # 2) ROI average in dB
    roi_features = []
    for roi_name, ch_idx in rois.items():
        roi_features.append(np.nanmean(bin_vals[ch_idx, :], axis=0))  # (n_bins,)
    roi_features = np.stack(roi_features, axis=1)  # (n_bins, n_rois)

    # 3) convert to relative power across bins (per ROI)
    roi_features = (roi_features - roi_features.mean(axis=0, keepdims=True)) / roi_features.std(axis=0, keepdims=True)

    roi_features_all[key] = roi_features

In [ ]:
df = pd.DataFrame([])
for key in roi_features_all.keys():
    df = pd.concat([df, pd.DataFrame(roi_features_all[key][0], columns=[key], index=list(rois.keys())).T], axis=0)

df.reset_index(inplace=True)
df[['bids_id', 'condition']] = df['index'].apply(lambda x: x.split('_')).apply(pd.Series)
df.drop(columns='index', inplace=True)
df['session'] = df['condition'].apply(lambda x: x[-1])

# open session data and merge with power values
session_data = pd.read_csv('data/behavioral_data/archived/behavioral_data.csv')
df = df.astype({'session': 'int64'})
df = pd.merge(session_data, df, how='right', on=['session', 'bids_id'], right_index=False)
df.insert(1, 'condition', df.pop('condition'))

## Classification